In [1]:
import cProfile
from breast_mri_qa import fetch, organise, measure

Define the Orthanc server information and login credentials.

In [2]:
host = '139.59.186.101'
port = 80
user = 'orthanc'
passwd = 'orthanc'

Create a 'Fetcher' object which can query data at the patient, study and series level as well as return an instance. 

In [3]:
fetcher = fetch.Fetcher(host=host, port=port, user=user, passwd=passwd)

Get the `n` most recent StudyUIDs of studies associated with a patient name

In [4]:
patient_name = 'BREAST*'
n_most_recent = 4
json_studies = fetcher.get_studies_json(patient_name=patient_name)
studies = []
for study in json_studies:
    c = {}
    c['StudyDate'] = study['00080020']['Value'][0]
    c['StudyUID'] = study['0020000D']['Value'][0]
    c['PatientName'] = study['00100010']['Value'][0]
    studies.append(c)
studies = sorted(studies, key=lambda k: k['StudyDate'], reverse=True)
studies = studies[:n_most_recent]
studyuids = [study['StudyUID'] for study in studies]
studyuids

[u'1.3.46.670589.11.42358.5.0.8420.2016100308225197001',
 u'1.3.46.670589.11.41453.5.0.5940.2016092608192762001',
 u'1.3.46.670589.11.42358.5.0.9980.2016091208201340001',
 u'1.3.46.670589.11.42358.5.0.5328.2016062712051513007']

Select the StudyUID you want y specifying the value of `selected_study` (remember they are zero-indexed). e.g. `studyuids[0]` for the first item, or `studyuids[1]` for the second item.

In [5]:
selected_study = 0
studyuid = studyuids[selected_study]
study = studies[selected_study]
studyuid

u'1.3.46.670589.11.42358.5.0.8420.2016100308225197001'

Get Series-level objects associated with this StudyUID.

In [ ]:
seriesuids = fetcher.get_series(studyuid)
seriesuids

[u'1.3.46.670589.11.42358.5.0.7188.2016100308231157000',
 u'1.3.46.670589.11.42358.5.0.4408.2016100308234269606',
 u'1.3.46.670589.11.42358.5.0.8548.2016100308243136000',
 u'1.3.46.670589.11.42358.5.0.4408.2016100308245260660',
 u'1.3.46.670589.11.42358.5.0.4408.2016100308262008687',
 u'1.3.46.670589.11.42358.5.0.4408.2016100308273083693',
 u'1.3.46.670589.11.42358.5.0.4408.2016100308234308608',
 u'1.3.46.670589.11.42358.5.0.4408.2016100308351203767',
 u'1.3.46.670589.11.42358.5.0.4408.2016100308352122802',
 u'1.3.46.670589.11.42358.5.0.4408.2016100308262013688',
 u'1.3.46.670589.11.42358.5.0.4408.2016100308352319826',
 u'1.3.46.670589.11.42358.5.0.4408.2016100308352520850',
 u'1.3.46.670589.11.42358.5.0.4408.2016100308353180955',
 u'1.3.46.670589.11.42358.5.0.4408.2016100308305153730',
 u'1.3.46.670589.11.42358.5.0.4408.2016100308353908012',
 u'1.3.46.670589.11.42358.5.0.4408.2016100308354486047',
 u'1.3.46.670589.11.42358.5.0.4408.2016100308354720082',
 u'1.3.46.670589.11.42358.5.0.4

Get the valid images associated with each of these series.

In [ ]:
instances = []
for uid in seriesuids:
    possible_instance = fetcher.get_valid_image_instance(studyuid, uid)
    if (possible_instance):
        instances.append(possible_instance)

Check whether the list of instances contains a full protocol. TODO The protocol is currently hard-coded - need to read in data at runtime!

In [ ]:
protocol = organise.Protocol()
missing_instances = protocol.assign_instances_to_protocol(instances)
#assert(missing_instances is None)
missing_instances

If you made if this far without an error then the study contains all the acquisitions required to do the QA - woohoo!

In [ ]:
def get_mid_slice(instance):
    pixel_array = instance['PixelArray']
    return pixel_array[int(len(pixel_array) / float(2))]

In [ ]:
images = protocol.dict_protocol_instances

In [ ]:
measure.calc_snr(get_mid_slice(images['snr_acquisition_one']), get_mid_slice(images['snr_acquisition_two']))

In [ ]:
measure.calc_fse(get_mid_slice(images['spir_fat']), get_mid_slice(images['spir_water']))

In [ ]:
measure.calc_fse(get_mid_slice(images['spair_fat']), get_mid_slice(images['spair_water']))

In [ ]:
num_coils = 7
num_to_str = {'1':'one','2':'two','3':'three','4':'four','5':'five','6':'six','7':'seven'}
coil_image_means = []
for i in range(1, num_coils + 1):
    c = {}
    identifier = str('coil_' + num_to_str[str(i)] + '_acquisition_one')
    c[identifier] = get_mid_slice(images[identifier]).mean()
    coil_image_means.append(c)
coil_image_means

In [ ]:
study